In [1]:
script_path = getwd()                                # %exclude_jupyterlab%
script_path                                          # %exclude_jupyterlab%

[1] "/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/scripts/rna"

In [2]:
#system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
#        '/sc_seurat_opt.R --genome="hg38" --data.dir=/Volumes/ccrsf-static/singlecell_projects/DwightNissley_CS035576_6scRNASeq_101223/Analysis_CPMV/Analysis/1_B3C2_NC/outs/filtered_feature_bc_matrix --outdir="test_dir"'),  # %exclude_jupyterlab%
#       intern = T) # %exclude_jupyterlab%

In [3]:
#system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
#        '/sc_seurat_opt.R --genome="hg38" --data.dir=/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/split_pipe_comb/1C0217/DGE_filtered --outdir="test_dir"'),  # %exclude_jupyterlab%
#       intern = T) # %exclude_jupyterlab%

In [4]:
system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
        '/sc_seurat_opt.R --genome="hg38" --data.dir=/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/split_pipe_comb/1V172/DGE_filtered/ --outdir="test_dir"'),  # %exclude_jupyterlab%


character(0)

In [5]:
library(Seurat)
library(Matrix)
library(MASS)
library(dplyr)
library(reshape2)
library(ggplot2)
library(URD)
library(cluster)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'gplots':
  method         from 
  reorder.factor gdata



In [6]:
opt = readRDS("./test_dir/opt.rds")                                 # %exclude_jupyterlab%

In [7]:
opt                                                              

$min.cells
[1] 1

$min.features
[1] 1

$data.dir
[1] "/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/split_pipe_comb/1V172/DGE_filtered/"

$sampleid
[1] "scRNA"

$genome
[1] "hg38"

$outdir
[1] "test_dir"

$help
[1] FALSE

In [8]:
system(paste0("mkdir -p ", opt$outdir))
setwd(opt$outdir) 

In [9]:
# Initialize variable (optional but explicit)
count_mtx <- NULL  

# Define file paths
parsebio_files <- c("count_matrix.mtx", "all_genes.csv", "cell_metadata.csv")
tenx_files <- c("matrix.mtx", "matrix.mtx.gz", 
                "features.tsv", "features.tsv.gz",
                "genes.tsv", "genes.tsv.gz",
                "barcodes.tsv", "barcodes.tsv.gz")

# Check file existence
has_parsebio <- all(file.exists(file.path(opt$data.dir, parsebio_files)))
has_10x <- any(file.exists(file.path(opt$data.dir, tenx_files)))

# Select reader
if (has_parsebio) {
  message("Detected ParseBio format.")
  count_mtx <- ReadParseBio(data.dir = opt$data.dir)
} else if (has_10x) {
  message("Detected 10x Genomics format.")
  count_mtx <- Read10X(data.dir = opt$data.dir)
} else {
  stop("No recognized data format found in: ", opt$data.dir)
}

Detected ParseBio format.



In [10]:
count_mtx <- count_mtx[(nrow(count_mtx)/2):nrow(count_mtx), 1:1000]                # %exclude_jupyterlab%

In [11]:
seur <- CreateSeuratObject(counts = count_mtx,
                                 min.cells = opt$min.cells,
                                 min.features = opt$min.features,
                                 project = opt$sampleid)

In [12]:
rm(count_mtx)

In [13]:
#find mitochondrial genes
if (startsWith(opt$genome, "mm10") | startsWith(opt$genome, "mm39")) {
        seur[["percent.mito"]] <- PercentageFeatureSet(seur, pattern="^mt-")
} else if (startsWith(opt$genome, "hg19") | startsWith(opt$genome, "hg38")) {
        seur[["percent.mito"]] <- PercentageFeatureSet(seur, pattern="^MT-")
}


In [14]:
head(seur@meta.data)
summary(seur@meta.data$percent.mito)                   # %exclude_jupyterlab%  
seur@meta.data[is.na(seur@meta.data$percent.mito), ]   # %exclude_jupyterlab%  

,orig.ident,nCount_RNA,nFeature_RNA,percent.mito
,<fct>,<dbl>,<int>,<dbl>
34_01_04__s1,34,14,14,0
34_01_09__s1,34,7,4,0
34_01_15__s1,34,9,6,0
34_01_34__s1,34,16,10,0
34_01_37__s1,34,6,4,0
34_01_42__s1,34,8,6,0


    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
 0.00000  0.00000  0.00000  0.02704  0.00000 16.66667 

orig.ident,nCount_RNA,nFeature_RNA,percent.mito
<fct>,<dbl>,<int>,<dbl>


In [34]:
summary(seur@meta.data$nCount_RNA)  # %exclude_jupyterlab%  
table(seur@meta.data$nCount_RNA)  # %exclude_jupyterlab%
summary(seur@meta.data$nFeature_RNA)  # %exclude_jupyterlab%  
table(seur@meta.data$nFeature_RNA)  # %exclude_jupyterlab% 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    6.00    8.00   10.22   12.00  332.00 


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
  9  27  64  67  82  90 103  84  83  75  60  41  36  37  23  17  16   8  10   9 
 21  22  23  24  25  26  27  29  31  32  33  35  36  39  47  49  52  56  80 196 
  9   5   9   6   6   3   1   2   1   3   2   1   1   1   1   1   1   1   1   1 
232 332 
  1   1 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    4.00    6.00    7.55    9.00  149.00 


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
 12  58  92 110 128 122 114  93  70  39  39  30  14  14  14   8   3   7   2   7 
 21  22  23  24  25  26  27  28  30  31  36  37  38  42 104 110 149 
  5   1   2   2   1   1   1   1   1   1   1   1   1   1   1   1   1 

In [18]:
png("VlnPlot_PreFilter.png", height=7, width=7, units='in', res=200)
VlnPlot(seur, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3)
dev.off()

Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”


png 
  2

In [19]:
plot1 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "percent.mito")
plot2 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
png("FeatureScatter_PreFilter.png", height=7, width=10, units='in', res=200)
plot1 | plot2
dev.off()

png 
  2

In [20]:
cS <- data.frame(libSize=seur$nCount_RNA, geneDetect=seur$nFeature_RNA)
p_hi <- 1e-3 #p-value for filtering doublets
p_lo <- 1e-2 #p-value for filtering poor libraries
fitLibSize <- fitdistr(cS$libSize,"negative binomial")
umi.upper.limit <- qnbinom(p_hi,size=fitLibSize$estimate["size"],
                 mu=fitLibSize$estimate["mu"],lower.tail=F)
umi.lower.limit <- qnbinom(p_lo,size=fitLibSize$estimate["size"],
                 mu=fitLibSize$estimate["mu"],lower.tail=T)
fitGeneDetect <- fitdistr(cS$geneDetect,"negative binomial")
gene.upper.limit <- qnbinom(p_hi,size=fitGeneDetect$estimate["size"],
                 mu=fitGeneDetect$estimate["mu"],lower.tail=F)
gene.lower.limit <- qnbinom(p_lo,size=fitGeneDetect$estimate["size"],
                 mu=fitGeneDetect$estimate["mu"],lower.tail=T)

cur.mad <- mad(seur$percent.mito)
cur.med <- median(seur$percent.mito)
diff.val <- 4 * cur.mad
mito.upper.limit <- cur.med + diff.val

temp_doublets <- (cS$libSize > umi.upper.limit) | (cS$geneDetect > gene.upper.limit) #doublets IDed based on high library size or genes detected
temp_crapLibs <- (cS$libSize < umi.lower.limit) | (cS$geneDetect < gene.lower.limit) #poor libraries IDed based on low library size or genes detected

Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”
Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”
Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”
Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”
Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”
Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”


In [21]:
print(mito.upper.limit)


[1] 0


In [22]:
#filter_summary <- t(data.frame(c("Doublets"=sum(temp_doublets), "Poor-Quality"=sum(temp_crapLibs), "Mitochondrial"=sum(seur$percent.mito > mito.upper.limit), "Total Filtered"=sum(temp_doublets | temp_crapLibs | seur$percent.mito > mito.upper.limit))))

In [38]:
filter_summary <- t(data.frame(c(
  "Doublets"=sum(temp_doublets),
  "Poor-Quality"=sum(temp_crapLibs),
  "Mitochondrial"=sum(seur$percent.mito >= mito.upper.limit),
  "Total Filtered"=sum(temp_doublets | temp_crapLibs | seur$percent.mito >= mito.upper.limit)
)))


In [39]:
dim(seur)

[1] 2520  999

In [40]:
rownames(filter_summary) = opt$sampleid
filter_summary

,Doublets,Poor-Quality,Mitochondrial,Total Filtered
scRNA,10,0,999,999


In [24]:
write.table(filter_summary, 'FilterNumbers.csv', sep=',', quote=FALSE, row.names=FALSE)

In [41]:
seur <- subset(seur, subset = nFeature_RNA >= gene.lower.limit & nFeature_RNA <= gene.upper.limit)
seur <- subset(seur, subset = nCount_RNA   >= umi.lower.limit  & nCount_RNA   <= umi.upper.limit)
# Only apply mitochondrial filtering if the threshold is positive
if (mito.upper.limit > 0) {
  seur <- subset(seur, subset = percent.mito <= mito.upper.limit)
}

In [ ]:
# seur@misc$qc_cutoffs <- list(
#   gene = c(lower = gene.lower.limit, upper = gene.upper.limit),
#   umi  = c(lower = umi.lower.limit,  upper = umi.upper.limit),
#   mito = mito.upper.limit
# )

In [ ]:
write.table(data.frame(umi.upper.limit, umi.lower.limit, gene.upper.limit, gene.lower.limit, mito.upper.limit), 
            'FilterThresholds.csv', sep=',', quote=FALSE, row.names = FALSE)

In [ ]:
png("VlnPlot_Filtered.png", height=7, width=7, units='in', res=200)
VlnPlot(seur, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3)
dev.off()

In [ ]:
#new plot
plot1 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "percent.mito")
plot2 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
png("FeatureScatter_PostFilter.png", height=7, width=10, units='in', res=300)
plot1 | plot2
dev.off()

In [ ]:
seur <- SCTransform(seur, vars.to.regress = "percent.mito", return.only.var.genes = FALSE, verbose = FALSE)

In [ ]:
saveRDS(seur, file = "seur_10x_preprocessed_object.rds")

In [ ]:
###URD
if (dim(seur)[[2]] > 10000) {
  numPCs <- 50
} else if (dim(seur)[[2]] < 500) {
  numPCs <- 10
} else {
  #inputTags <- as.matrix(read.csv(expressionFile, row.names = 1))
  mat1 <- as.matrix(GetAssayData(seur))
  cat("createURD...")
  test <- suppressWarnings(createURD(count.data = mat1, min.cells=3, min.counts=3)) # )
  cat("calcPCA...")
  test <- suppressWarnings(calcPCA(test, mp.factor = 2))
  write.table(test@pca.sig,"URD.txt")
  png("URD.png", height=7, width=7, units='in', res=300)
  pcSDPlot(test)
  dev.off()

  numPCs <- max(10, sum(test@pca.sig))
}

In [ ]:
top10 <- head(VariableFeatures(seur), 10)
# plot variable features with and without labels
plot1 <- VariableFeaturePlot(seur)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
png('VariableFeatures.png', height=7, width=10, units='in', res=300)
plot1 | plot2
dev.off()

In [ ]:
seur <- RunPCA(seur, 
               features = VariableFeatures(seur), 
               npcs=max(20,numPCs), do.print = TRUE, 
               ndims.print = 1:5, nfeatures.print = 5)

In [ ]:
pdf("VizPCAPlot.pdf")
for (i in  seq(1,20,2)) {
  j = i + 1
  print(VizDimLoadings(seur, i:j))
}
dev.off()

In [ ]:
#plot PCA
pdf("AllPCAPlot.pdf")
for (i in c(1:10)) {
  print(DimPlot(seur, dims=c(i, i+1), reduction="pca"))
}
dev.off()

In [ ]:
#PC heatmap
pdf("PC_HeatmapPlot.pdf")
for (i in c(1:10)) {
  DimHeatmap(seur, dims = i, cells = 500, balanced = TRUE)
}
dev.off()

#make PC elbow plot
pdf("PC_ElbowPlot.pdf")
ElbowPlot(seur)
dev.off()

In [ ]:
resolutions <- c(0.1, 0.3, 0.6, 0.8)

seur <- FindNeighbors(seur, dims=1:numPCs)
seur <- RunTSNE(seur, dims=1:numPCs)
write.csv(Embeddings(seur, reduction='tsne'), file = "tSNECoordinates.csv")
seur <- RunUMAP(seur, dims=1:numPCs)
write.csv(Embeddings(seur, reduction='umap'), file = "UMAPCoordinates.csv")

In [ ]:
runRes <- c()
tsnePlots <- list()
umapPlots <- list()
for (res in resolutions) {
  seur <- FindClusters(seur, dims=1:numPCs, resolution = res, print.output = 0, save.SNN = T)
  tsne <- TSNEPlot(seur) + ggtitle(paste(numPCs,"PCs_res", res, sep="")) +
    theme(plot.title = element_text(hjust = 0.5))
  tsnePlots[[as.character(res)]] <- tsne
  png(paste("TSNEPlotwith",numPCs,"PCs_", res, ".png", sep=""), height=7, width=7, units='in', res=300)
  print(tsne)
  dev.off()
  umap <- DimPlot(seur, reduction="umap") + ggtitle(paste(numPCs,"PCs_res", res, sep="")) +
    theme(plot.title = element_text(hjust = 0.5))
  umapPlots[[as.character(res)]] <- umap
  png(paste("UMAPPlotwith",numPCs,"PCs_", res, ".png", sep=""), height=7, width=7, units='in', res=300)
  print(umap)
  dev.off()

  try({
    seur.markers <- FindAllMarkers(object = seur, logfc.threshold = 0.25, only.pos=TRUE)
    write.csv(seur.markers %>% group_by(cluster) %>% top_n(-100,
                                                           p_val), paste("top100markers_pc", numPCs, "_res", res, ".csv", sep = ""))
    saveRDS(seur.markers, paste("markers_res", res, ".rds", sep = ""))
    runRes <- append(runRes, res)})
}



In [ ]:
#save object
saveRDS(seur, file = "seur_10x_cluster_object.rds")

In [ ]:
pdf("TSNEPlots.pdf")
for (res in tsnePlots){
  print(res)
}
dev.off()

In [ ]:
pdf("UMAPPlots.pdf")
for (res in umapPlots){
  print(res)
}
dev.off()

In [ ]:
##Create Silhoutte Plots
for (res in runRes){
  coord <- Embeddings(seur, reduction.type='pca')[,1:numPCs]
  Idents(seur) <- seur@meta.data[[paste0('SCT_snn_res.', res)]]
  clusters <- Idents(seur)
  d <- dist(coord, method="euclidean")
  sil<-silhouette(as.numeric(clusters), dist=d)
  #silPlot <- recordPlot()
  pdf(paste0("SilhouettePlot_res",res,".pdf"))#, height=7, width=7, units='in', res=300)
  plot(sil, col=as.factor(clusters[order(clusters, decreasing=FALSE)]), main=paste("Silhouette plot of Seurat clustering - resolution ", res, sep=""), lty=2)
  abline(v=mean(sil[,3]), col="red4", lty=2)
  dev.off()
}

##Remove resolutions that failed marker generation
print(runRes)
for (res in setdiff(resolutions, runRes)){
  seur@meta.data[paste0('SCT_snn_res.', res)] <- NULL
}

write(min(runRes), "minRes.txt")

In [ ]:
sessionInfo()

In [4]:
notebook_prefix = "sc_seurat"                                                         # %exclude_jupyterlab%
notebook_name = paste0(notebook_prefix, ".ipynb")                                     # %exclude_jupyterlab%
notebook_r = paste0(script_path, "/", paste0(notebook_prefix, ".r"))                  # %exclude_jupyterlab%
notebook_path = paste0(script_path, "/", notebook_name)                               # %exclude_jupyterlab%
opt_name = paste0(script_path, "/", sub(".ipynb", "_opt.R", notebook_name))           # %exclude_jupyterlab%
output = paste0(script_path, "/", sub(".ipynb", ".prod.R", notebook_name))            # %exclude_jupyterlab%
cmd1 = paste0("jupyter nbconvert --to script --output ",                              # %exclude_jupyterlab%
             notebook_prefix, ' ', notebook_path, "> /dev/null 2>&1 ")                # %exclude_jupyterlab%
cmd1                                                                                  # %exclude_jupyterlab%
system(cmd1, intern = TRUE)                                                           # %exclude_jupyterlab%

[1] "jupyter nbconvert --to script --output sc_seurat /mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/scripts/rna/sc_seurat.ipynb> /dev/null 2>&1 "

character(0)

In [5]:
cmd2 = paste0('cat -s ', opt_name, ' ', notebook_r,                                   # %exclude_jupyterlab%
             ' |grep -v exclude_jupyterlab > ', output,  ' 2>&1')                     # %exclude_jupyterlab%
cmd2                                                                                  # %exclude_jupyterlab%
system(cmd2, intern = T)                                                              # %exclude_jupyterlab%
system(paste0("rm ", notebook_r))                                                     # %exclude_jupyterlab%  

[1] "cat -s /mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/scripts/rna/sc_seurat_opt.R /mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/scripts/rna/sc_seurat.r |grep -v exclude_jupyterlab > /mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/scripts/rna/sc_seurat.prod.R 2>&1"

character(0)